## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [3]:
#!ls -al  ../../Data/Weather/

In [4]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [5]:
#(Res_eig['stations'])

In [6]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,-0.208829,-0.081790,0.034514,-0.043394
1,-0.219086,-0.136681,0.010381,-0.099503
2,-0.214883,-0.097468,0.031663,-0.007368
3,-0.209167,-0.143286,0.006438,-0.121900
4,-0.236098,-0.105204,0.033633,-0.038001


In [7]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00029622', u'USW00053019', u'US1AZMR0218', u'USC00029534', u'USC00026653']


,0,1,2,3
station,,,,
USC00029622,-0.208829,-0.081790,0.034514,-0.043394
USW00053019,-0.219086,-0.136681,0.010381,-0.099503
US1AZMR0218,-0.214883,-0.097468,0.031663,-0.007368
USC00029534,-0.209167,-0.143286,0.006438,-0.121900
USC00026653,-0.236098,-0.105204,0.033633,-0.038001


In [8]:
table=Eig.join(Stations,how='left')

In [9]:
#table=table[['name','latitude','longitude','elevation',0,1,2,3,4,5,6,7]]  #Original line 
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00029622,YOUNG,34.1000,-110.9333,1539.8,-0.208829,-0.081790,0.034514,-0.043394
USW00053019,WHITERIVER A 1 SW,33.8208,-109.9831,1576.7,-0.219086,-0.136681,0.010381,-0.099503
US1AZMR0218,GILBERT 3.6 ENE,33.3671,-111.7204,395.3,-0.214883,-0.097468,0.031663,-0.007368
USC00029534,WORKMAN CREEK 1,33.8167,-110.9167,2126.0,-0.209167,-0.143286,0.006438,-0.121900


In [10]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [11]:
#min_lat,max_lat,min_long,max_long = box = (42.1103, 42.6167, -72.6, -70.8)

min_lat,max_lat,min_long,max_long = box = (33.3767, 34.2639, -111.3447, -108.5167)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9

url1 = 'http://{s}.tile.opentopomap.org/{z}/{x}/{y}.png'
url2 = 'http://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png'
topoLayer = TileLayer(url=url1, opacity=0.6)
m = Map(default_tiles=topoLayer, center=center, zoom=zoom)
#m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)

r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.